In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%cd /content/drive/My Drive/MyProjects/Transformers

/content/drive/My Drive/MyProjects/Transformers


In [3]:
ls "."

attention_visual.ipynb  config.py          model.py           requirements.txt   train_.py
Beam_Search.ipynb       dataset.py         model_train.ipynb  runs/              train_wb.py
Colab_Train.ipynb       Inference.ipynb    __pycache__/       tokenizer_en.json  translate.py
conda.txt               Local_Train.ipynb  README.md          tokenizer_it.json  troubleshoot.ipynb


In [4]:
%%capture
!pip install datasets
!pip install tokenizers
!pip install torchmetrics

In [5]:
# import shutil

# def delete_directory(path):
#     try:
#         shutil.rmtree(path)
#         print(f"The directory at {path} has been deleted successfully.")
#     except Exception as e:
#         print(f"Failed to delete the directory: {e}")

# # Usage
# delete_directory('/content/.config')

In [6]:
import torch
! pip install matplotlib
from train_ import run_validation,get_model,Tokenizer,build_transformer,get_ds
from config import get_config
from pathlib import Path
from torch.utils.data import DataLoader,Dataset
from datasets import load_dataset
from torch.utils.data import random_split
from dataset import BilingualDataset
from datasets import load_dataset


/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [7]:
config = get_config()
print(config)

{'batch_size': 8, 'num_epochs': 20, 'lr': 0.0001, 'seq_len': 350, 'd_model': 512, 'datasource': 'opus_books', 'lang_src': 'en', 'lang_tgt': 'it', 'model_folder': '/content/drive/MyDrive/Models/pytorch-transformer/weights', 'model_basename': 'tmodel_', 'preload': 'latest', 'tokenizer_file': 'tokenizer_{0}.json', 'experiment_name': 'runs/tmodel'}


In [8]:
source_tokenizer_path = str(Path( "/content/drive/My Drive/Models/pytorch-transformer/vocab/tokenizer_en.json"))
dest_tokenizer_path = str(Path( "/content/drive/My Drive/Models/pytorch-transformer/vocab/tokenizer_it.json"))
print(source_tokenizer_path,dest_tokenizer_path)

/content/drive/My Drive/Models/pytorch-transformer/vocab/tokenizer_en.json /content/drive/My Drive/Models/pytorch-transformer/vocab/tokenizer_it.json


In [9]:
tokenizer_src = Tokenizer.from_file(source_tokenizer_path)
tokenizer_tgt = Tokenizer.from_file(dest_tokenizer_path)

In [10]:
device =  "cpu"
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
model.to(device)

Transformer(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attention_block): MultiHeadAttentionBlock(
          (w_q): Linear(in_features=512, out_features=512, bias=False)
          (w_k): Linear(in_features=512, out_features=512, bias=False)
          (w_v): Linear(in_features=512, out_features=512, bias=False)
          (w_o): Linear(in_features=512, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward_block): FeedForwardBlock(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (residual_connections): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
    )
    (norm): LayerNormalization

In [11]:
model_filename = "/content/drive/My Drive/Models/pytorch-transformer/tmodel_3.pt"
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
print(model)

In [ ]:

def get_data(config,tokenizer_src,tokenizer_tgt):
    # It only has the train split, so we divide it overselves
    ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

    # Build tokenizers
    tokenizer_src = tokenizer_src
    tokenizer_tgt = tokenizer_tgt

    # Keep 90% for training, 10% for validation
    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    # Find the maximum length of each sentence in the source and target sentence
    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')


    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=8, shuffle=True)

    return train_dataloader,val_dataloader


In [ ]:
train_data,val_dataloader = get_data(config,tokenizer_src,tokenizer_tgt)

In [ ]:
import torch
import torch.nn as nn
import torchmetrics
import os
import matplotlib.pyplot as plt
from train_ import greedy_batch_decode
from dataset import causal_mask
import torch.nn.functional as F

In [ ]:
cer = []
wer = []
bleu = []
val_losses = []



# def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
#     sos_idx = tokenizer_tgt.token_to_id('[SOS]')
#     eos_idx = tokenizer_tgt.token_to_id('[EOS]')

#     # Precompute the encoder output and reuse it for every step
#     encoder_output = model.encode(source, source_mask)
#     # Initialize the decoder input with the sos token
#     decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
#     while True:
#         if decoder_input.size(1) == max_len:
#             break

#         # build mask for target
#         decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

#         # calculate output
#         out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

#         # get next token
#         prob = model.project(out[:, -1])
#         _, next_word = torch.max(prob, dim=1)
#         decoder_input = torch.cat(
#             [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
#         )

#         if next_word == eos_idx:
#             break

#     result = decoder_input.squeeze(0)
#     return result




def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device, return_logits=False):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)

    # Optionally collect logits for loss computation
    logits_list = []

    while True:
        if decoder_input.size(1) >= max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # project to vocabulary
        prob = model.project(out[:, -1])  # Ensure this outputs logits
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, next_word.unsqueeze(0).unsqueeze(0)], dim=1
        )

        # Collect logits if needed for loss calculation
        if return_logits:
            logits_list.append(prob.unsqueeze(1))

        if next_word.item() == eos_idx:
            break

    if return_logits:
        # Concatenate along sequence length dimension
        logits = torch.cat(logits_list, dim=1)
        return logits
    else:
        return decoder_input.squeeze(0)



# def greedy_batch_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
#     # Decode each input sequence in the batch individually
#     results = [greedy_decode(model, src.unsqueeze(0), mask.unsqueeze(0), tokenizer_src, tokenizer_tgt, max_len, device) for src, mask in zip(source, source_mask)]

#     # Define a fixed maximum length (350) for padding
#     fixed_max_length = max_len

#     # Pad each decoded sequence to the fixed maximum length with the PAD token
#     padded_results = [F.pad(result, (0, fixed_max_length - result.size(0)), 'constant', tokenizer_tgt.token_to_id('[PAD]')) for result in results]

#     # Stack the padded sequences into a single tensor for batch processing
#     results_tensor = torch.stack(padded_results)
#     print("Shape after padding to fixed max length:", results_tensor.shape)
#     return results_tensor


def greedy_batch_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    # Decode each input sequence in the batch individually and collect logits
    results = [greedy_decode(model, src.unsqueeze(0), mask.unsqueeze(0), tokenizer_src, tokenizer_tgt, max_len, device, return_logits=True) for src, mask in zip(source, source_mask)]

    # Determine the maximum sequence length from the logits for uniform padding
    max_length = max(result.size(1) for result in results)

    # Pad each decoded logits tensor to the maximum sequence length
    padded_results = [F.pad(result, (0, 0, 0, max_length - result.size(1)), 'constant', tokenizer_tgt.token_to_id('[PAD]')) for result in results]

    # Stack the padded logits tensors into a single tensor for batch processing
    # results_tensor = torch.stack(padded_results)
    # print("Shape after padding to fixed max length:", results_tensor.shape)
    # return results_tensor
    for data in padded_results:
        print(data.shape)
    return
    return torch.stack(padded_results)

# This function will now correctly return a tensor of shape [batch_size, seq_len, vocab_size]
# where seq_len is the maximum sequence length in the batch, and vocab_size is the number of tokens.

def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device):

    model.eval()
    total_loss = 0
    metric_cer = torchmetrics.CharErrorRate()
    metric_wer = torchmetrics.WordErrorRate()
    metric_bleu = torchmetrics.BLEUScore()

    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
    source_texts, expected, predicted = [], [], []


    try:
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)
            target = batch["label"].to(device) # (b, seq_len)



            model_out = greedy_batch_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device) # (b, seq_len)
            # loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
            # lloss = loss_fn(model_out.view(-1, tokenizer_tgt.get_vocab_size()), target.view(-1))
            print("Model output shape:", model_out.shape)
            print("Model traget shape:",target.shape)
            print("Reshaped model output for loss calculation:", model_out.view(-1, tokenizer_tgt.get_vocab_size()).shape)
            loss = loss_fn(model_out.view(-1, tokenizer_tgt.get_vocab_size()), target.view(-1))
            total_loss += loss.item() * encoder_input.size(0)  # Multiply by batch size for accurate average



            for idx in range(encoder_input.size(0)):
                source_text = tokenizer_src.decode(encoder_input[idx].cpu().numpy())
                target_text = tokenizer_tgt.decode(target[idx].cpu().numpy())
                predicted_text = tokenizer_tgt.decode(model_out[idx].cpu().numpy())
                print(f"Source: {source_text}")
                print(f"Target: {target_text}")
                print(f"Predicted: {predicted_text}")
                print("-" * console_width)

                source_texts.append(source_text)
                expected.append(target_text)
                predicted.append(predicted_text)




    # Calculate and log metrics
    avg_loss = total_loss / len(validation_ds.dataset)
    val_losses.append(avg_loss)
    cer_data = metric_cer(predicted, expected)
    wer_data = metric_wer(predicted, expected)
    bleu_data = metric_bleu(predicted, expected)
    cer.append(cer_data)
    wer.append(wer_data)
    bleu.append(bleu_data)


    print(f'Validation Loss: {avg_loss:.4f}, CER: {cer_data:.4f}, WER: {wer_data:.4f}, BLEU: {bleu_data:.4f}')
    return avg_loss, cer, wer, bleu




In [ ]:
average_loss,cer,wer,bleu = run_validation(model,val_dataloader,tokenizer_src,tokenizer_tgt,config["seq_len"],device)

In [ ]:
for batch in val_dataloader:
    print(batch["encoder_input"].shape)
    print(batch["encoder_mask"].shape)
    print(batch["label"].shape)
    break

350